In [1]:
from ift6758.client.serving_client import ServingClient
from ift6758.client.game_client import GameClient

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
Game_client = GameClient()
Serving_client = ServingClient()
workspace = "ift-6758-projet-quipe-13"
model = 'xgboost-best-select-features'
version = '1.0.0'

print(Game_client.gameId)
print(Game_client.game_ended)
game_id = 2021020329 # int or str ??
df = Game_client.process_query(game_id, model_name=model) # model_name="..."
print(df)



out = Serving_client.download_registry_model(workspace, model, version)
out


# pred = Serving_client.predict(df)
# pred


# print(Game_client.gameId)
# Game_client.gameId = game_id
# print(Game_client.game_ended)
# print(df)

# df1 = Game_client.process_query(game_id) # model_name="..."
# print(df1)



0
None
    periodTimeSec  period  coordinate_x  coordinate_y  distanceFromGoal  \
0            62.0       1          56.0          17.0         37.121422   
1           110.0       1         -65.0         -10.0         26.000000   
2           170.0       1          78.0          16.0         19.416488   
3           202.0       1          84.0         -36.0         36.345564   
4           249.0       1         -39.0           8.0         50.635956   
5           350.0       1          82.0          29.0         29.832868   
6           385.0       1         -74.0          16.0         21.931712   
7           417.0       1         -53.0         -26.0         44.407207   
8           457.0       1         -25.0         -17.0         66.219333   
9           471.0       1          55.0         -30.0         45.343136   
10          562.0       1         -47.0         -20.0         46.518813   
11          568.0       1         -89.0         -13.0        178.474088   
12          624.0 

<Response [200]>

In [4]:
df.columns

Index(['periodTimeSec', 'period', 'coordinate_x', 'coordinate_y',
       'distanceFromGoal', 'shotAngle', 'lastEventCoord_x', 'lastEventCoord_y',
       'timeDifference', 'distanceDifference', 'rebound',
       'shotAngleDifference', 'speed', 'Backhand', 'Deflected', 'Slap Shot',
       'Snap Shot', 'Wrist Shot', 'BLOCKED_SHOT', 'FACEOFF', 'GIVEAWAY', 'HIT',
       'MISSED_SHOT', 'SHOT', 'TAKEAWAY', 'Tip-In', 'Wrap-around', 'GOAL'],
      dtype='object')

In [4]:
features = ['Feature1', 'Feature2','Feature3','Feature4','Feature5',
            'Feature6', 'Feature7', 'Feature8', 'Feature9', 'Feature10',
            'Feature11', 'Feature12', 'Feature13', 'Feature14', 'Feature15',
            'Feature16', 'Feature17', 'Feature18']
feature_values = np.random.rand(5, 18) 
model_pred = np.random.rand(5,1)

df2 = pd.DataFrame(feature_values, columns=features)
df2['Model Output'] = model_pred


df2 = pd.concat([df2, None])

df2

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Model Output
0,0.338336,0.658229,0.080374,0.653884,0.998830,0.595151,0.990930,0.763072,0.325790,0.316878,0.643384,0.746079,0.622047,0.864508,0.490822,0.469827,0.285092,0.309956,0.099495
1,0.863446,0.931487,0.482349,0.983587,0.051569,0.407327,0.491138,0.656567,0.209392,0.515680,0.133533,0.691370,0.992321,0.901606,0.741880,0.865251,0.488679,0.914833,0.957006
2,0.912018,0.549632,0.324200,0.373318,0.355033,0.550976,0.597387,0.276959,0.259340,0.972029,0.148174,0.601377,0.537072,0.371170,0.969081,0.462712,0.860787,0.248386,0.607115
3,0.683397,0.501065,0.625312,0.786553,0.364606,0.903314,0.900569,0.876023,0.762679,0.903298,0.855247,0.613760,0.162374,0.948730,0.069152,0.475184,0.486452,0.746251,0.641119
4,0.712955,0.332575,0.424619,0.594863,0.402931,0.809193,0.242383,0.293519,0.875471,0.192501,0.965750,0.514316,0.508247,0.248859,0.688297,0.546752,0.440919,0.358854,0.898370


In [4]:
try:
    hey = requests.get("http://0.0.0.0:5000/hello")
except Exception as e:
    print(e)

In [5]:
hey.text

'Hello!'

In [7]:
# default_model = {
#     'workspace': 'ift-6758-projet-quipe-13',
#     'model': 'xgboost-best-all-features',
#     'version': '1.0.0'
# }
# model = requests.post("http://127.0.0.1:8080/download_registry_model", json=default_model)
# model.text

'"Updated from local folder."\n'

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
pred.text

'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'

<Response [200]>

In [13]:
%xmode Verbose
%tb

Exception reporting mode: Verbose


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().